# 基于MindSpore构造卷积层

本实验主要专注于卷积层的原理和构造，使用MindSpore构造卷积层。

## 1、实验目的

- 掌握卷积层的原理和特征。
- 掌握如何使用MindSpore构造卷积层。

## 2、卷积层原理介绍

卷积层的主要作用是提取图像中的特征，浅层卷积层用来提取边缘、纹理等特征，深层卷积层用来提取更加抽象的特征，整合在一起构成了丰富的图像特征。

卷积层由一组和输入层有着相同通道数的卷积核构成，将卷积核在输入层上按照指定步长滑动，每到一个位置计算卷积核权重和对应输入特征的乘积，所有乘积求和作为输出特征图的一个特征值，单个卷积核在输入特征上滑动一遍就得到了单通道的输出特征，一组卷积核全部进行上述过程就得到了包含若干通道的输出特征图。这种每次滑动到一个局部位置的计算方式能够在非常大的程度上减少参数量，同时一个物体无论在图像中的什么位置，这一组卷积核都能够提取其特征，具有平移不变性。

以单通道为例，卷积的简易过程如图所示，从左到右三部分依次为输入图像、卷积核（过滤器）、输出特征，输入图像尺寸为5×5，卷积核尺寸为3×3，输出特征图尺寸为3×3。卷积核中心滑动到输入图像中某一像素的位置，则重叠的部分分别计算乘积，最后求和作为当前位置的输出特征值。
<img src="./Figures/fig001.jpg" style="zoom:70%" />

图中的卷积计算过程如下：

第一行卷积的结果为：1×(-1) + 0×(-1) + 0×(-1) = -1 

第二行卷积的结果为：1×(-1) + 0× 1 + 1×(-1) = -2

第三行卷积的结果为：1×(-1) + 0×(-1) + 1×(-1) = -2

然后把三行的结果相加等于-5。

卷积层输出特征图尺寸计算方式：输入特征图尺寸为$w_i×h_i×c_i$（分别为宽、高、通道数），卷积核的尺寸为$k_i×k_i$（k表示卷积核的边长），输出通道数为$n_{i+1}$，卷积步长为$s$，输入特征图周围填充0特征值的行数为$p_0$，输出特征图的尺寸为$w_{i+1}×h_{i+1}×c_{i+1}$，则输出特征图的尺寸为：
$$w_{i+1}=\frac{w_i-k_i+2p_0}{s}+1$$
$$h_{i+1}=\frac{h_i-k_i+2p_0}{s}+1$$
$$c_{i+1}=n_{i+1}$$

当卷积结束之后，可以发现图像变小了，原来是5×5的，现在变成了3×3的，这不是想要的结果，因此进行零填充，因为填充的是二维图像，所以在图像四周都要填成0。填充完之后再开始做卷积运算，中间部分已得出，需要计算新填充的部分，用同样的方式计算完所有的位置，二维卷积的零填充如图所示：
<img src="./Figures/fig002.jpg" style="zoom:70%" />
原来的图像是5×5的，零填充之后，得到的也是5×5，大小保持不变。
根据上面的公式，我们可以计算输出特征图的尺寸。输入特征图尺寸为5×5×1，卷积核的尺寸为3×3，卷积步长$s$为1，输入特征图周围填充0特征值的行数$p_0$为1，根据公式得到输出特征图的尺寸为5×5×1。
$$w_{i+1}=\frac{5-3+2}{1}+1=5$$
$$h_{i+1}=\frac{5-3+2}{1}+1=5$$
$$c_{i+1}=n_{i+1}=1$$

## 3、实验环境

在动手进行实践之前，需要注意以下几点：
* 确保实验环境正确安装，包括安装MindSpore。安装过程：首先登录[MindSpore官网安装页面](https://www.mindspore.cn/install)，根据安装指南下载安装包及查询相关文档。同时，官网环境安装也可以按下表说明找到对应环境搭建文档链接，根据环境搭建手册配置对应的实验环境。
* 推荐使用交互式的计算环境Jupyter Notebook，其交互性强，易于可视化，适合频繁修改的数据分析实验环境。
* 实验也可以在华为云一站式的AI开发平台ModelArts上完成。
* 推荐实验环境：MindSpore版本=MindSpore 2.0；Python环境=3.7


|  硬件平台 |  操作系统  | 软件环境 | 开发环境 | 环境搭建链接 |
| :-----:| :----: | :----: |:----:   |:----:   |
| CPU | Windows-x64 | MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.1节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| GPU CUDA 10.1|Linux-x86_64| MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.2节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| Ascend 910  | Linux-x86_64| MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第四章](./MindSpore环境搭建实验手册.docx)|

## 4、数据处理

### 4.1 数据准备

对输入Tensor计算二维卷积。该Tensor的常见shape为$(N,C_{in},H_{in},W_{in})$，其中N为batch size，$C_{in}$为空间维度，$H_{in}$,$W_{in}$分别为特征层的高度和宽度。现给出Tensor的shape为(1,120,1024,640)计算二维卷积。

### 4.2 数据加载

导入需要的Python库，调用mindspore.nn.Conv2d()，它是对输入Tensor计算二维卷积，其中has_bias设置为默认值'False'，weight_init设置为'normal'，然后用给出的Tensor进行数据加载，Tensor的shape为(1,120,1024,640)，然后输出。

In [71]:
from mindspore.common.tensor import Tensor
import mindspore.nn as nn
import numpy as np
import mindspore.common.dtype as mstype

# 给出Tensor的shape为(1,120,1024,640)计算二维卷积。
net = nn.Conv2d(120, 240, 4, has_bias=False, weight_init='normal')
# 输入
x = Tensor(np.ones([1, 120, 1024, 640]), mstype.float32)
output = net(x)
# 输出
print(output[:,6])
print(output.shape)

[[[-0.17673276  0.01873722  0.01873722 ...  0.01873722  0.15733194
    0.18270826]
  [-0.02393796  0.1633252   0.1633252  ...  0.1633252   0.23819993
    0.22824927]
  [-0.02393796  0.1633252   0.1633252  ...  0.1633252   0.23819993
    0.22824927]
  ...
  [-0.02393796  0.1633252   0.1633252  ...  0.1633252   0.23819993
    0.22824927]
  [ 0.214774    0.27904704  0.27904704 ...  0.27904704  0.29368705
    0.23236483]
  [ 0.19655114  0.28352883  0.28352883 ...  0.2835287   0.36032274
    0.21884324]]]
(1, 240, 1024, 640)


## 5、模型构建

**导入Python库&模块**

在使用前，导入需要的Python库。

In [72]:
# 导入科学计算库
import numpy as np
# 导入神经网络模块
import mindspore.nn as nn
# 被初始化的Tensor的数据类型
import mindspore.common.dtype as mstype
from mindspore import log as logger
# 环境设置模块
from mindspore import context
# 常见操作
from mindspore.ops import operations as P
from mindspore.common.parameter import Parameter
# 导入初始化神经元的各个参数
from mindspore.common.initializer import initializer
# 用于初始化Tensor的Tensor
from mindspore import Tensor
import mindspore._checkparam as Validator # nightly测试
# from mindspore._checkparam import twice
from mindspore._extends import cell_attr_register 
# MindSpore中神经网络的基本构成单元
from mindspore.nn.cell import Cell

**使用Mindspore官方定义的卷积神经网络层**

使用MindSpore官方定义的_Conv类，该类首先定义卷积过程中要使用的参数，如：卷积核的高度和宽度、卷积核的移动步长、填充模式以及数据格式等参数。然后初始化卷积层。

In [73]:
# 对由多个输入平面组成的输入信号应用N-D卷积
class _Conv(Cell):
    def __init__(self,
                 in_channels,        # 输入Tensor的空间维度
                 out_channels,       # 输出Tensor的空间维度
                 kernel_size,        # 指定卷积核的高度和宽度
                 stride,             # 卷积核的移动步长
                 pad_mode,           # 指定填充模式。默认值"same"，指输出的高度和宽度分别与输入整除stride后的值相同。
                 padding,            # 输入的高度和宽度方向上填充的数量
                 dilation,           # 卷积核膨胀尺寸
                 group,              # 将过滤器拆分为组
                 has_bias,           # 卷积层是否添加偏置参数，默认值：False
                 weight_init,        # 权重参数的初始化方法
                 bias_init,          # 偏置参数的初始化方法
                 data_format='NCHW', # 数据格式的可选值有"NHWC"、"NCHW"。默认值:"NCHW"
                 transposed=False):
        # 初始化卷积层
        super(_Conv, self).__init__()
        # 对于构造函数中的self参数，其代表的是当前正在初始化的类对象
        self.in_channels = Validator.check_positive_int(in_channels, 'in_channels', self.cls_name)
        self.out_channels = Validator.check_positive_int(out_channels, 'out_channels', self.cls_name)
        self.kernel_size = kernel_size
        # self所表示的是实际调用该方法的对象
        self.stride = stride
        self.pad_mode = pad_mode
        self.weight_init = weight_init
        self.bias_init = bias_init
        # 数据格式的可选值有"NHWC"、"NCHW"。默认值:"NCHW"
        self.data_format = Validator.check_string(data_format, ['NCHW', 'NHWC', 'NCDHW'], 'format', self.cls_name)
        if context.get_context("device_target") != "GPU" and self.data_format == "NHWC":
            raise ValueError(f"For '{self.cls_name}', the \"NHWC\" format only support in GPU target, "
                             f"but got the 'format' is {self.data_format} and "
                             f"the platform is {context.get_context('device_target')}.")
        if isinstance(padding, int):
            Validator.check_non_negative_int(padding, 'padding', self.cls_name)
            self.padding = padding
        elif isinstance(padding, tuple):
            # 对输入进行填充。在输入的高度和宽度方向上填充padding大小的0
            for pad in padding:
                Validator.check_non_negative_int(pad, 'padding item', self.cls_name)
            self.padding = padding
        else:
            raise TypeError(f"For '{self.cls_name}', the type of 'padding' must be int or tuple(int), "
                            f"but got {type(padding).__name__}.")

        # 卷积核膨胀尺寸
        self.dilation = dilation
        self.group = Validator.check_positive_int(group)
        # 卷积层是否添加偏置参数
        self.has_bias = has_bias
        # 卷积核的高度和宽度。数据类型为整型或两个整型的tuple
        for kernel_size_elem in kernel_size:
            Validator.check_positive_int(kernel_size_elem, 'kernel_size item', self.cls_name)
        for stride_elem in stride:
            Validator.check_positive_int(stride_elem, 'stride item', self.cls_name)
        for dilation_elem in dilation:
            Validator.check_positive_int(dilation_elem, 'dilation item', self.cls_name)
        # 将过滤器拆分为组
        if in_channels % group != 0:
            raise ValueError(f"For '{self.cls_name}', the attr 'in_channels' must be divisible by attr 'group', "
                             f"but got 'in_channels': {in_channels} and 'group': {group}.")
        # in_channels和out_channels必须可被group整除
        # 如果组数等于in_channels和out_channels，这个二维卷积层也被称为二维深度卷积层。默认值：1。
        if out_channels % group != 0:
            raise ValueError(f"For '{self.cls_name}', the 'out_channels' must be divisible by attr 'group', "
                             f"but got 'out_channels': {out_channels} and 'group': {group}.")
        if transposed:
            shape = [in_channels, out_channels // group, *kernel_size]
        else:
            shape = [out_channels, *kernel_size, in_channels // group] \
                if self.data_format == "NHWC" else [out_channels, in_channels // group, *kernel_size]
        # 权重参数
        self.weight = Parameter(initializer(self.weight_init, shape), name='weight')

        if Validator.check_bool(has_bias, "has_bias", self.cls_name):
            self.bias = Parameter(initializer(self.bias_init, [out_channels]), name='bias')
        else:
            if self.bias_init != 'zeros':
                logger.warning("Value of 'has_bias' is False, value of 'bias_init' will be ignored.")
            self.bias = None

    # 构造网络
    def construct(self, *inputs):
        raise NotImplementedError

    def extend_repr(self):
        s = 'input_channels={}, output_channels={}, kernel_size={}, ' \
            'stride={}, pad_mode={}, padding={}, dilation={}, ' \
            'group={}, has_bias={}, ' \
            'weight_init={}, bias_init={}, format={}'.format(
                # self所表示的是实际调用该方法的对象
                self.in_channels,
                self.out_channels,
                self.kernel_size,
                self.stride,
                self.pad_mode,
                self.padding,
                self.dilation,
                self.group,
                self.has_bias,
                self.weight_init,
                self.bias_init,
                self.data_format)
        return s

**实现二维卷积运算**

定义Conv2d类，首先进行初始化，该过程包括初始化参数和初始化二维卷积两部分，初始化参数部分将二维卷积核的移动步长定义为1，填充模式设置为默认值'same'，二维卷积核膨胀尺寸设置为1等。然后初始化二维卷积，最后构造二维卷积神经网络。

In [74]:
class Conv2d(_Conv):
    @cell_attr_register
    def __init__(self,
                 in_channels,         # 输入Tensor的空间维度
                 out_channels,        # 输出Tensor的空间维度
                 kernel_size,         # 指定二维卷积核的高度和宽度
                 stride=1,            # 二维卷积核的移动步长
                 pad_mode='same',     # 指定填充模式。默认值"same"，指输出的高度和宽度分别与输入整除stride后的值相同。
                 padding=0,           # 输入的高度和宽度方向上填充的数量
                 dilation=1,          # 二维卷积核膨胀尺寸
                 group=1,             # 将过滤器拆分为组
                 has_bias=False,      # 二维卷积层是否添加偏置参数，默认值：False
                 weight_init='normal',# 权重参数的初始化方法
                 bias_init='zeros',   # 偏置参数的初始化方法
                 data_format='NCHW'): # 数据格式的可选值有"NHWC"、"NCHW"。默认值:"NCHW"
        # 初始化二维卷积
        kernel_size = Validator.twice(kernel_size) # nightly测试
        stride = Validator.twice(stride)
        self._dilation = dilation
        dilation = Validator.twice(dilation)
        super(Conv2d, self).__init__(
            in_channels,
            out_channels,
            kernel_size,
            stride,
            pad_mode,
            padding,
            dilation,
            group,
            has_bias,
            weight_init,
            bias_init,
            data_format)
        self.conv2d = P.Conv2D(out_channel=self.out_channels,
                               kernel_size=self.kernel_size,
                               mode=1,
                               pad_mode=self.pad_mode,
                               pad=self.padding,
                               stride=self.stride,
                               dilation=self.dilation,
                               group=self.group,
                               data_format=self.data_format)
        self.bias_add = P.BiasAdd(data_format=self.data_format)

    # 构造二维卷积神经网络
    def construct(self, x):
        output = self.conv2d(x, self.weight)
        if self.has_bias:
            output = self.bias_add(output, self.bias)
        return output

## 6、模型测试

给定Tensor的shape为(1,120,1024,640)，然后调用Conv2d函数来计算二维卷积，最后输出结果。

In [60]:
net = Conv2d(120, 240, 4, has_bias=False, weight_init='normal')
# 输入
x = Tensor(np.ones([1, 120, 1024, 640]), mstype.float32)
# 输出
output = net(x)
print(output[:,6])
print(output.shape)

[[[ 0.02908076  0.05783416  0.05783416 ...  0.05783416  0.0747276
   -0.10985402]
  [-0.18333124 -0.2664398  -0.2664398  ... -0.2664398  -0.16775273
   -0.19988897]
  [-0.18333124 -0.2664398  -0.2664398  ... -0.2664398  -0.16775273
   -0.19988897]
  ...
  [-0.18333124 -0.2664398  -0.2664398  ... -0.2664398  -0.16775273
   -0.19988897]
  [-0.23326604 -0.3525596  -0.3525596  ... -0.3525596  -0.24423432
   -0.22144832]
  [-0.34975982 -0.29625094 -0.29625094 ... -0.29625088 -0.09688124
   -0.07446267]]]
(1, 240, 1024, 640)
